# Immigrant Project
### Cross-Generational Differences in Hispanic Outcomes by Gender

*Including #1 as the instructions for Gender (#2) are similar to #1*
1.	Cross-Generational Differences in Hispanic Outcomes, 1994-2016: use main_national again, and plot the values for white-All; Hispanic-All; Hispanic, 1st; Hispanic, 2nd; and Hispanic, 3rd for each outcome listed above other than gen1-gen3. 

2.	Cross-Generational Differences in Hispanic Outcomes by Gender, 1994-2016: similar to (1) broken down by gender, side-by-side would be great other than gen1-gen3.

In [1]:
import pandas as pd
import os
import plotly.graph_objs as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

In [2]:
os.chdir(r'H:\CALDER\CALDER Data Visualizations\Data\Immigrant Project')

In [3]:
gender = pd.read_csv('main_national_gender.csv')

In [4]:
gender = gender.sort_values(by=['year', 'wbhao', 'igen'])
gender.head(9)

,year,female,wbhao,igen,lths,college,hinsured,employed,married2,children,gen1,gen2,gen3,rincp_all
6,1994,0,Hispanic,1st Generation,0.509119,0.107574,0.520085,0.836146,0.728057,0.681814,NaN,NaN,NaN,22640.2710
7,1994,1,Hispanic,1st Generation,0.501592,0.089247,0.610190,0.505678,0.647664,0.773266,NaN,NaN,NaN,9112.9834
10,1994,0,Hispanic,2nd Generation,0.257090,0.111617,0.709334,0.762288,0.630529,0.540059,NaN,NaN,NaN,32315.5450
11,1994,1,Hispanic,2nd Generation,0.285207,0.124322,0.803798,0.580352,0.531698,0.641696,NaN,NaN,NaN,16248.2560
14,1994,0,Hispanic,3rd Generation,0.198040,0.101136,0.746446,0.874805,0.660013,0.622343,NaN,NaN,NaN,38778.6520
15,1994,1,Hispanic,3rd Generation,0.174399,0.091921,0.815824,0.689799,0.639064,0.694427,NaN,NaN,NaN,19389.3260
18,1994,0,Hispanic,All,0.385189,0.106980,0.611512,0.829152,0.691921,0.638198,0.561195,0.212924,0.225881,26395.3380
19,1994,1,Hispanic,All,0.375840,0.097781,0.702207,0.565829,0.619505,0.725064,0.539341,0.225370,0.235288,12771.1040
4,1994,0,White,1st Generation,0.126657,0.429245,0.799556,0.858250,0.763655,0.546997,NaN,NaN,NaN,48521.7930


In [5]:
# Rename Columns
pre = ['lths', 'college', 'hinsured', 'employed', 'married2', 'children', 'gen1', 'gen2', 'gen3','rincp_all']
post = ["% less than High School diploma", "% with College Degree", "% with Health Insurance", "% Employed", "% Married", 
        "% with Children", "Share of 1st Generation", "Share of 2nd Generation", "Share of 3rd Generation", 
        "Median Individual Real Income"]
for i in range(0, len(pre)):   
    gender.rename(columns={pre[i]: post[i]}, inplace=True)

gender.columns

Index(['year', 'female', 'wbhao', 'igen', '% less than High School diploma',
       '% with College Degree', '% with Health Insurance', '% Employed',
       '% Married', '% with Children', 'Share of 1st Generation',
       'Share of 2nd Generation', 'Share of 3rd Generation',
       'Median Individual Real Income'],
      dtype='object')

In [6]:
gender['wbhao_igen'] = gender['wbhao'] + ' ' + gender['igen']

### Drop Columns to match Umut's request in #1

In [7]:
gender = gender.drop(gender[(gender.wbhao_igen == 'White 1st Generation') | (gender.wbhao_igen == 'White 2nd Generation')
                       | (gender.wbhao_igen == 'White 3rd Generation')].index)

In [8]:
gender = gender.drop(['Share of 1st Generation','Share of 2nd Generation', 'Share of 3rd Generation'], axis=1)

In [9]:
gender.head(8)

,year,female,wbhao,igen,% less than High School diploma,% with College Degree,% with Health Insurance,% Employed,% Married,% with Children,Median Individual Real Income,wbhao_igen
6,1994,0,Hispanic,1st Generation,0.509119,0.107574,0.520085,0.836146,0.728057,0.681814,22640.2710,Hispanic 1st Generation
7,1994,1,Hispanic,1st Generation,0.501592,0.089247,0.610190,0.505678,0.647664,0.773266,9112.9834,Hispanic 1st Generation
10,1994,0,Hispanic,2nd Generation,0.257090,0.111617,0.709334,0.762288,0.630529,0.540059,32315.5450,Hispanic 2nd Generation
11,1994,1,Hispanic,2nd Generation,0.285207,0.124322,0.803798,0.580352,0.531698,0.641696,16248.2560,Hispanic 2nd Generation
14,1994,0,Hispanic,3rd Generation,0.198040,0.101136,0.746446,0.874805,0.660013,0.622343,38778.6520,Hispanic 3rd Generation
15,1994,1,Hispanic,3rd Generation,0.174399,0.091921,0.815824,0.689799,0.639064,0.694427,19389.3260,Hispanic 3rd Generation
18,1994,0,Hispanic,All,0.385189,0.106980,0.611512,0.829152,0.691921,0.638198,26395.3380,Hispanic All
19,1994,1,Hispanic,All,0.375840,0.097781,0.702207,0.565829,0.619505,0.725064,12771.1040,Hispanic All


In [24]:
gender['wbhao_igen'].unique()

array(['Hispanic 1st Generation', 'Hispanic 2nd Generation',
       'Hispanic 3rd Generation', 'Hispanic All', 'White All'], dtype=object)

## Graph

In [27]:
app = dash.Dash()

app.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"}) 

df = gender

outcomes = ["% less than High School diploma", "% with College Degree", "% with Health Insurance", "% Employed", 
            "% Married", "% with Children", "Median Individual Real Income"]

# Organize where items will be on the page
app.layout = html.Div([
        html.H3(
            children='Cross-Generational Differences in Hispanic Outcomes by Gender, 1994-2016',
            style={
                'textAlign': 'center', 'fontFamily' : 'Georgia'
            }
        ),
        html.Center([          
            html.Div([
                html.Div([html.P('Select Outcome',id='outcome-title')],
                    style={'textAlign': 'center', 'fontFamily': 'Georgia'}),
                dcc.Dropdown(
                    id='outcome-id',
                    options=[{'label': i, 'value': i} for i in outcomes],
                    value='% less than High School diploma')
                ],style={'width': '50%','textAlign': 'center', 'fontFamily': 'Georgia', 'display': 'inline-block'}),        
            ]),
            html.Div([
                html.Div([
                    dcc.Graph(id='indicator-graphic1',
                              config={'modeBarButtonsToRemove': ['sendDataToCloud', 'lasso2d', 'zoomIn2d', 'zoomOut2d', 'pan2d', 
                                                                 'zoom2d','resetScale2d'], 
                                    'displaylogo': False})
                    ], style={'width': '50%', 'display': 'inline-block'}),  
                html.Div([
                    dcc.Graph(id='indicator-graphic2',
                              config={'modeBarButtonsToRemove': ['sendDataToCloud', 'lasso2d', 'zoomIn2d', 'zoomOut2d', 'pan2d', 
                                                                 'zoom2d','resetScale2d'], 
                                    'displaylogo': False})
                    ], style={'width': '50%', 'display': 'inline-block'}),             
            ]),
    ])
@app.callback(

    dash.dependencies.Output('indicator-graphic1', 'figure'),
    [dash.dependencies.Input('outcome-id', 'value')])
def outcome_male(outcome_id):
    dff = df[['year', 'wbhao_igen', 'female',outcome_id]]
    dff = dff[(dff.female == 0)]

    lines = {}
    data = []
    
    m_max = dff[outcome_id].max()
    m_min = dff[outcome_id].min()
    f = df[['year', 'wbhao_igen', 'female',outcome_id]]
    f = f[(f.female == 1)]
    f_max = f[outcome_id].max()
    f_min = f[outcome_id].min()
    if m_max>f_max:
        df_max = m_max
    elif m_max<f_max:
        df_max = f_max
    if m_min>f_min:
        df_min = f_min
    elif m_min<f_min:
        df_min = m_min
    if df_min<0.05:
        df_min = 0
        
    y_axis = {'title': '{0}'.format(outcome_id), 
              'hoverformat': ',.2f',
              'range': [df_min, df_max]}          
    legends={'orientation': 'h', 'xanchor': 'center', 'x': '0.5', 'y': '-0.22'}

    
    # Show three lines for each output
    generation = ['White All','Hispanic All', 'Hispanic 1st Generation', 'Hispanic 2nd Generation', 
                  'Hispanic 3rd Generation']
    for gen in generation:
        if '1st' in gen:
             lines = dict(
                 color = ("#6b6ecf"),
                 width = 3)
        if '2nd' in gen:
             lines = dict(
                 color = ("#80b1d3"),
                 width = 3)
        if '3rd' in gen:
             lines = dict(
                 color = ("#fdb462"),
                 width = 3)
        if 'White All' in gen:
              lines = dict(
                 color = ("#333333"),
                 width = 3)
        if 'Hispanic All' in gen:
               lines = dict(
                 color = ("#333333"),
                 width = 3,
                 dash = 'dot')
        trace = go.Scatter(
            x = dff[(dff.wbhao_igen == gen)]['year'],
            y = dff[(dff.wbhao_igen == gen)][outcome_id],
            mode='lines',
            name = gen,
            line = lines,
            opacity = 0.8
            )      
        data.append(trace)

    return {
        'data' : data,
        'layout' : go.Layout(
            title='Male',
            titlefont=dict(
                        family='Georgia'),
            xaxis={'title': 'Year'},
            yaxis=y_axis,
            legend = legends
        )
    }

@app.callback(

    dash.dependencies.Output('indicator-graphic2', 'figure'),
    [dash.dependencies.Input('outcome-id', 'value')])
def outcome_female(outcome_id):
    dff = df[['year', 'wbhao_igen', 'female',outcome_id]]
    dff = dff[(dff.female == 1)]

    lines = {}
    data = []
    
    f_max = dff[outcome_id].max()
    f_min = dff[outcome_id].min()
    m = df[['year', 'wbhao_igen', 'female',outcome_id]]
    m = m[(m.female == 0)]
    m_max = m[outcome_id].max()
    m_min = m[outcome_id].min()
    if f_max>m_max:
        df_max = f_max
    elif f_max<m_max:
        df_max = m_max
    if f_min>m_min:
        df_min = m_min
    elif f_min<m_min:
        df_min = f_min
    if df_min<0.05:
        df_min = 0

    y_axis = {'title': '{0}'.format(outcome_id), 
              'hoverformat': ',.2f',
              'range': [df_min, df_max]}          
    legends={'orientation': 'h', 'xanchor': 'center', 'x': '0.5', 'y': '-0.22'}

    
    # Show three lines for each output
    generation = ['White All','Hispanic All', 'Hispanic 1st Generation', 'Hispanic 2nd Generation', 
                  'Hispanic 3rd Generation']
    for gen in generation:
        if '1st' in gen:
             lines = dict(
                 color = ("#6b6ecf"),
                 width = 3)
        if '2nd' in gen:
             lines = dict(
                 color = ("#80b1d3"),
                 width = 3)
        if '3rd' in gen:
             lines = dict(
                 color = ("#fdb462"),
                 width = 3)
        if 'White All' in gen:
              lines = dict(
                 color = ("#333333"),
                 width = 3)
        if 'Hispanic All' in gen:
               lines = dict(
                 color = ("#333333"),
                 width = 3,
                 dash = 'dot')
        trace = go.Scatter(
            x = dff[(dff.wbhao_igen == gen)]['year'],
            y = dff[(dff.wbhao_igen == gen)][outcome_id],
            mode='lines',
            name = gen,
            line = lines,
            opacity = 0.8
            )      
        data.append(trace)

    return {
        'data' : data,
        'layout' : go.Layout(
            title = 'Female',
            titlefont=dict(
                        family='Georgia'),
            xaxis={'title': 'Year'},
            yaxis=y_axis,
            legend = legends
        )
    }
    
if __name__ == '__main__':
    app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Apr/2018 10:37:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2018 10:37:45] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2018 10:37:45] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2018 10:37:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2018 10:37:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2018 10:37:45] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2018 10:38:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2018 10:38:07] "POST /_dash-update-component HTTP/1.1" 200 -
